# Домашнее задание №4. Качество данных

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression

Будем предсказывать цену квартиры в зависимости от различных параметров

In [19]:
# Создаём сэмпл
n_samples = 1000
# Параметры квартиры
price_m2 = np.random.choice(60000, n_samples) + 80000
center_dist_km = np.random.random_sample((n_samples,)) * 15
metro_dist_km = np.random.random_sample((n_samples,)) * 2
parking = np.random.choice(2, n_samples)
floor = np.random.choice(20, n_samples) + 1
square = np.random.choice(44, n_samples) + 16
# Итоговая цена квартиры
total_price = price_m2 * square - center_dist_km * 100000 + parking * 100000

data = pd.DataFrame({'price_m2': price_m2,
                     'center_dist_km': center_dist_km,
                     'metro_dist_km': metro_dist_km,
                     'parking': parking,
                     'floor': floor,
                     'square': square,
                     'total_price': total_price})
data.head(5)

,price_m2,center_dist_km,metro_dist_km,parking,floor,square,total_price
0,95660,1.327771,1.718041,0,17,34,3.119663e+06
1,99730,6.232464,0.002294,1,7,25,1.970004e+06
2,128789,12.385201,0.899257,1,20,31,2.853939e+06
3,114837,4.990313,0.080808,1,6,39,4.079612e+06
4,132921,11.171273,0.287931,0,14,53,5.927686e+06


1. Попробуем предсказать цену на основе всех параметров, за исключением целевой переменной.

In [20]:
from sklearn.metrics import mean_absolute_error

X = data.drop(columns=['total_price'])
y = data['total_price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(X)
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 3.75093884e+01 -1.01981417e+05  1.80325890e+03  1.12805102e+05
 -2.95708127e+03  1.08499354e+05]
Bias: -4028460.4469225216
Error: 167670.7346865691


2. Модель плохо справилась с задачей: несмотря на то, что модель достаточно точно нашла коэффициенты для параметров center_dist_km и parking, она подобрала очень большие коэффициенты для "лишних" параметров. Попробуем убрать значения параметров, которых нет в формуле для определения цены квартиры, и заново обучить модель.

In [22]:
X = data.drop(columns=['total_price', 'metro_dist_km', 'floor'])
y = data['total_price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(X)
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 3.75205626e+01 -1.01891148e+05  1.13203258e+05  1.08617314e+05]
Bias: -4064018.625716708
Error: 167951.79706571382


3. Удаление лишних столбцов не помогло улучшить качество модели. Добавим столбец, который является произведением двух других столбцов, т.к. наша модель не может сделать этого.

In [25]:
data['mult'] = data['price_m2'] * data['square']
X = data.drop(columns=['total_price', 'metro_dist_km', 'floor', 'price_m2',
                       'square'])
y = data['total_price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(X)
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-1.e+05  1.e+05  1.e+00]
Bias: 0.00101566594094038
Error: 0.00032686198875308035


**Вывод:** чтобы модель линейной регрессии показывала хорошие результаты, необходимо учитывать взаимосвязь переменных, характеризующих исследуемый объект. Возможно, если бы наши данные не были случайным образом сгенерированы, модель сумела бы отыскать скрытые связи между переменными (подобрать коэффициенты более точно).